<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/myGPTlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# myGPTlab

This app helps to process texts (longform content) with ChatGPT via API.

## Start:
**Run all cells - that's it.**
- The notebooks checks automatically, if an initial setup (with PIP etc.) is necessary. The setup status is saved in the file 'installation.done'
- You can force the setup by deleting this file or by going to Seciton "Setup and Configuration" and checking "inital_setup_mode". Afterwards uncheck "Setup and Configuration".

## Working with myGPTlab

## FAQ
### Was tun, wenn kryptische Fehlermeldungen erscheinen, wie z. B. "error: invalid group reference 11 at position 1"?
Den Input-Text neu anlegen bzw. konvertieren (z. b. per pandoc). Die Ursache kann Encoding sein.




Here we go

In [1]:
# @title
import os
import requests
# os.environ.pop('CREDS') # delete environment variable for debugging
if not os.getenv('CREDS'):
  #popup('Enter API Keys as JSON:')
  fname = 'aknip-colab-setup'
  url = 'https://bit.ly/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run aknip-colab-setup
else:
  print("API Keys available in env 'CREDS'")

!pip install ipywidgets -q

Secrets (JSON string): ··········
Credentials stored in env 'CREDS'. Get values via: 
creds = json.loads(os.getenv('CREDS')) 
key = creds['OpenAI']['v1']['credential']
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00


In [2]:
# @title
import ipywidgets
from ipywidgets import Layout, Button, Box, VBox, HTML, Output, Textarea

# Info flex proportionally to the weight
info_1 = HTML(value='<div style="background-color: red; text-align: center"><b>Please wait for initial setup...</b></div>', layout=Layout(flex='1 1 auto', width='auto'))
items_info = [info_1]

# Header flex proportionally to the weight
header_1 = HTML(value='<div style="background-color: #ccc; text-align: center">Hello <b>World</b></div>', layout=Layout(flex='1 1 auto', width='auto'))
items_header = [header_1]

# Text flex proportionally to the weight
text_1 = HTML(value='Infotext', layout=Layout(flex='1 1 auto', width='25%'))
text_2 = HTML(value='Infotext jdskl fjkdslf jsdklfjslf', layout=Layout(flex='1 1 auto', width='25%'))
text_3 = HTML(value='Infotext jdskl fjkdslf jsdklfjslf', layout=Layout(flex='1 1 auto', width='25%'))
text_4 = HTML(value='Infotext jdskl fjkdslf jsdklfjslf', layout=Layout(flex='1 1 auto', width='25%'))
items_texts = [text_1, text_2, text_3, text_4]

# Textinput flex proportionally to the weight
input_1 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
input_2 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
input_3 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
input_4 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
items_inputs = [input_1, input_2, input_3, input_4]

# Buttons flex proportionally to the weight
button_1 = Button(description='Summarize Articles', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
button_2 = Button(description='Save Text', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
button_3 = Button(description='Split Text', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
button_4 = Button(description='Chat', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
items_buttons = [button_1, button_2, button_3, button_4]

# Output flex proportionally to the weight
output_1 = Output(layout=Layout(flex='1 1 auto', width='auto'))
items_output = [output_1]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='90%')

box_info = Box(children=items_info, layout=box_layout)
box_header = Box(children=items_header, layout=box_layout)
box_texts = Box(children=items_texts, layout=box_layout)
box_inputs = Box(children=items_inputs, layout=box_layout)
box_buttons = Box(children=items_buttons, layout=box_layout)
box_output = Box(children=items_output, layout=box_layout)

def on_button_1_clicked(b):
    #with output_1:
    comment = 'Kommentar zum Projekt...'
    execute_project('Handelsblatt', 'gpt-3.5-turbo', '', promptlib_global, 'summary-multiple-articles', '1', 'length-max-dyn', '', 0 ,True, folders, input_1.value, comment)
button_1.on_click(on_button_1_clicked)

def on_button_2_clicked(b):
    output_1.clear_output()

button_2.on_click(on_button_2_clicked)

#button_1.layout.visibility = "hidden"
#button_2.layout.visibility = "hidden"
#button_3.layout.visibility = "hidden"
#button_4.layout.visibility = "hidden"

VBox([box_info, box_header, box_texts, box_inputs, box_buttons, box_output])


# Install

In [3]:
# @title Settings

# @markdown Default model
default_model = 'GPT-3.5' # @param ["GPT-3.5", "GPT-4"]

# @markdown Start Gradio webapp.
start_gradio_webapp = False # @param {type:"boolean"}

# @markdown Initial Setup Mode for pip install, fetch credentials etc.
initial_setup_mode = False # @param {type:"boolean"}

# @markdown Debug Mode for extensive logging.
debug_mode = True # @param {type:"boolean"}

# @markdown iOS Mode to develop helper functions, no Gradio.
# @markdown Useful for development on iOS, eg. with Carnets App
ios_mode = False # @param {type:"boolean"}

In [4]:
folders = {
    'audio': 'audio',
    'audio-chunks': 'audio/chunks',
    'transcript':'audio-transcript',
    'transcript-chunks': 'audio-transcript/chunks',
    'text-input': 'text-input',
    'text-input-backup': 'text-input-backup',
    'text-output': 'text-output',
    'text-output-logs': 'text-output-logs'
}

In [5]:
from ipywidgets import widgets
from IPython.display import Javascript, display, clear_output
notify_output = widgets.Output()
display(notify_output)
@notify_output.capture()
def popup(text):
    clear_output()
    display(Javascript("alert('{}')".format(text)))
#popup('Hello World!')

Output()

In [6]:
import os
if initial_setup_mode != True:
  if os.path.exists('installation.done'):
      initial_setup_mode = False
      print('No initial setup - forced by existing file "installation.done"')
  else:
    initial_setup_mode = True
    print('Starting automatic setup - forced by missing file "installation.done".\n\nEnter API Keys as JSON (in next notebook cell).')
    #popup('Starting automatic setup. Enter API Keys as JSON (in next notebook cell).')
else:
  print('Starting setup.\n\nEnter API Keys as JSON (in next notebook cell).')
  #popup('Starting setup. Enter API Keys as JSON (in next notebook cell).')

Starting automatic setup - forced by missing file "installation.done".

Enter API Keys as JSON (in next notebook cell).


In [7]:
creds = json.loads(os.getenv('CREDS'))
# openAI_key = creds['OpenAI']['v2']['credential']
# print(openAI_key)

In [8]:
if initial_setup_mode == True:
  !pip install openai==0.27.7 yt-dlp==2023.7.6 librosa==0.10.0.post2 pickle-mixin==1.0.2 langchain==0.0.225 PyPDF2==3.0.1 PyMuPDF==1.22.5 pypandoc==1.11 gradio -q
  %load_ext gradio
else:
  print('No initial setup.')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.9 MB/s eta 0:00:

In [9]:
if initial_setup_mode == True:
  f= open('installation.done','w+')
  f.close()
  print('Initial setup done. Application starting.')
  # popup('Initial setup done. Application starting.')
else:
  print('No initial setup.')

Initial setup done. Application starting.


In [97]:
import requests

def import_prompt_lib():
  fname = 'prompt_lib.ipynb'
  url = 'https://raw.githubusercontent.com/aknip/Streamlit-Gradio/main/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run prompt_lib.ipynb

def import_helper_lib():
  fname = 'helper_lib.ipynb'
  url = 'https://raw.githubusercontent.com/aknip/Streamlit-Gradio/main/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run helper_lib.ipynb

In [98]:
import_helper_lib()
import_prompt_lib()

In [12]:
print(promptlib_global)
print(testfunc(3,4))

{'summary-shortened': {'description': '', 'category': 'summarize', '1': {'note': 'Gleichzeitiges Einfügen/Formatieren von Headlines funktioniert nicht, lässt GPT alles umformulieren           ("Format headings and subheadings in markdown syntax.").           Test S4/HANA Kap. 4: Input 12.300 Wörter, GPT-3.5 Output: 4.500 Wörter (36%), GPT-4 Output: 3.400 Wörter (27%)', 'prompt': 'Paraphrase the following text and shorten each paragraph of the paraphrased text to maxiumum the half of its original length by focussing            on the key aspects. Write in the style and tone of a professional business analyst and use passive form. Be detailed and precise            and use every content detail of the text. Note that there may be typographical or grammatical errors in the passages due to the            text being fetched from a transcript of a talk or a speech.            {language_text} {length_text} Text:```{input_text}``` {toc_text}', 'lang-de': 'Write everything in German language.', 

In [13]:
# @title
test_text1 = 'Die Photosynthese ist ein physiologischer Prozess zur Erzeugung energiereicher Biomoleküle aus energieärmeren Stoffen mit Hilfe\
  von Lichtenergie. Sie wird von Pflanzen, Algen und manchen Bakterien betrieben. Bei diesem biochemischen Vorgang wird Lichtenergie mit Hilfe\
  von lichtabsorbierenden Farbstoffen wie Chlorophyll in chemische Energie umgewandelt. Diese wird dann genutzt, um aus energiearmen\
  anorganischen Stoffen (vor allem Kohlenstoffdioxid (CO2) und Wasser (H2O)) energiereiche organische Verbindungen (vor allem Kohlenhydrate)\
  aufzubauen. Der genutzte Anteil der eingestrahlten Energie, nämlich der zum Aufbau der Assimilate verwendete Anteil, wird photosynthetische\
  Effizienz genannt. Soweit die energiereichen organischen Stoffe zu Bestandteilen des Lebewesens werden, bezeichnet man deren Synthese als\
  Assimilation. Man unterscheidet zwischen oxygener und anoxygener Photosynthese. Bei der oxygenen Photosynthese wird molekularer\
  Sauerstoff (O2) freigesetzt. Bei der anoxygenen Photosynthese, die nur von Bakterien betrieben wird, entstehen statt Sauerstoff andere\
  anorganische Stoffe, beispielsweise elementarer Schwefel (S). Die Photosynthese ist der einzige biochemische Prozess, bei dem\
  Lichtenergie, meistens Sonnenlicht, in chemisch gebundene Energie umgewandelt wird (Phototrophie). Indirekt sind auch fast alle\
  heterotrophen (nicht zur Photosynthese fähigen) Lebewesen von ihr abhängig, da sie der Photosynthese letztlich ihre Nahrung und auch\
  den zur Energiegewinnung mittels aerober Atmung benötigten Sauerstoff verdanken. Aus dem Sauerstoff entsteht außerdem die schützende\
  Ozonschicht der Erdatmosphäre.'
test_text2 = f'''
# The Potential of AI in Education

So, anyone who's been paying attention for the last few months has been seeing headlines like this, especially in education. The thesis has been students are going to be using chat GPT and other forms of AI to cheat, do their assignments, they're not going to learn, and it's going to completely undermine education as we know it.

Now what I'm going to argue today is not only are there ways to mitigate all of that, if we put the right guardrails, we do the right things, we can mitigate it, but I think we're at the cusp of using AI for probably the biggest positive transformation that education has ever seen. And the way we're going to do that is by giving every student on the planet an artificially intelligent but amazing personal tutor, and we're going to give every teacher on the planet an amazing, artificially intelligent teaching assistant.  And just to appreciate how big of a deal it would be to give everyone a personal tutor, I show you this clip from Benjamin Bloom's 1984 two-sigma study, or he called it the two-sigma problem.

The two-sigma comes from two standard deviations, sigma the symbol for standard deviation, and he had good data that showed that, look, a normal distribution, that's the one that you see in the traditional bell curve right in the middle, that's how the world kind of sorts itself out, that if you were to give personal one-to-one tutoring for students, that you could actually get a distribution that looks like that right, it says tutorial one-to-one with the asterisks, like that right distribution, a two standard deviation improvement. Just to put that in plain language, that could take your average student and turn them into an exceptional student, it can take your below average student and turn them into an above average student.

# The Challenge of Scaling Personalized Instruction

Now, the reason why he framed it as a problem was he said, well, this is all good, but how do you actually scale group instruction this way? How do you actually give it to everyone in an economic way? What I'm about to show you is, I think, the first moves towards doing that. Obviously, we've been trying to approximate it in some way at Khan Academy for over a decade now, but I think we're at the cusp of accelerating it dramatically.  I'm going to show you the early stages of what RAI, which we call Khan Migo, what it can now do, and maybe a little bit of where it is actually going.  This right over here is a traditional exercise that you or many of your children might have seen on Khan Academy, but what's new is that little bot thing at the right, and we'll start by seeing one of the very important safeguards, which is the conversation is recorded and viewable by your teacher.  It's moderated, actually, by a second AI, and also, it does not tell you the answer.  It is not a cheating tool.  Notice, when the student says, tell me the answer, it says, I'm your tutor.

What do you think is the next step for solving the problem? Now, if the student makes a mistake, and this will surprise people who think large language models are not good at mathematics, notice not only does it notice the mistake, it asks the student to explain their reasoning, but it's actually doing what I would say not just even an average tutor would do, but an excellent tutor would do. It's actually able to divine what is probably the misconception in that student's mind, that they probably didn't use the distributive properly.  Remember, we need to distribute the negative two to both the nine and the 2M inside of the parentheses.  This to me is a very, very, very big deal, and it's not just in math.  This is a computer programming exercise on Khan Academy where the student needs to make the clouds part, and so we can see the student starts defining a variable, left X minus minus.

# AI as a Super Tutor

It only made the left cloud part, but then they can ask a con amigo, what's going on? Why is only the left cloud moving? And it understands the code. It knows all the context of what the student is doing, and it understands that those ellipses are there to draw clouds, which I think is kind of mind-blowing, and it says, to make the right cloud move as well, try adding a line of code inside the draw function that increments the right X variable by one pixel in each frame.  Now, this one is maybe even more amazing, because we have a lot of math teachers.  We've all been trying to teach the world to code, but there aren't a lot of computing teachers out there, and what you just saw, even when I'm tutoring my kids when they're learning to code, I can't help them this well, this fast.  This is really going to be a super tutor.  And it's not just exercises.  It understands what you're watching.

It understands the context of your video. It can answer the age-old question, why do I need to learn this? And it asks, socratically, well, what do you care about? And let's say the student says, I want to be a professional athlete, and it says, well, learning about the size of cells, which is what this video is about, that could be really useful for understanding nutrition and how your body works, et cetera.  It can answer questions.  It can quiz you.  It can connect it to other ideas.  You can now ask as many questions of a video as you could ever dream of.  So another big shortage out there, I remember the high school I went to, the student-to-guidance counselor ratio was about 200 or 300 to 1.  A lot of the country, it's worse than that.  We can use Conmigo to give every student a guidance counselor, academic coach, career coach, life coach, which is exactly what you see right over here.  And this is all, you know, we launched this with the GPT-4 launch.  We have a few thousand people on this.  This isn't a fake demo.

§§§

# Conversations with Historical Figures and AI Debates

This is really in action. And then there's, you know, things that I think it would have been even harder, it would have been a little science fiction to do with even a traditional tutor.  There's a student, we run an online high school with Arizona State University called Khan World School.  And we have a student who attends that online school based in India, her name is Sanvi, and she was doing a report on the Great Gatsby.  And when she was reading the Great Gatsby, Jay Gatsby keeps looking at the green light off into the distance.  And she's like, well, why does he do that? She did some web searches, and people have obviously studied this and commented about the symbolism of that, but none of it was really resonating with her.  And then she realized that she had Conmigo and that she could talk to Jay Gatsby himself.

# Unlocking Learning Literature and AI Debates

And so, ah, splendid choice, old sport, I am now Jay Gatsby, the enigmatic millionaire from Escott Jarrell's classic. And so why do you keep staring at the green light? Ah, the green light, old sport.  It's a symbol of my dreams and desires, you see.  It's situated at the end of Daisy Buchanan's dock, across the bay from my mansion.  I gaze at it longingly as it represents my yearning for the past and my hope to reunite with Daisy, the love of my life.  And what was cool is Sanvi had said, I had this long conversation, she called him Mr.  Gatsby.  And at the end, she actually apologized for taking his time, which I thought was very polite of her.  But you can imagine, this unlocks learning literature, learning, you can talk to historical figures, we're even probably going to add an activity, you can talk to, like, the Mississippi River.  It brings things to life in ways that really were science fiction even six months or a year ago.  Students can get into debates with the AI.  And we've got, here's the students debating whether we should cancel student debt.  The student is against canceling student debt.  And we've gotten very clear feedback.  We've started running it at Con World School and our lab school that we have, Con Lab School.  The students, the high school students especially, they're saying, this is amazing to be able to fine tune my arguments without fearing judgment.

# Enhancing Language Arts with AI Writing Collaboration

It makes me that much more confident to kind of go into the classroom and really participate. And we all know that Socratic dialogue debate is a great way to learn.  But frankly, it's not out there for most students.  But now it can be accessible to hopefully everyone.  A lot of the narrative, we saw that in the headlines, has been it's going to do the writing for kids.  Kids are not going to learn to write.  But we are showing that there's ways that the AI doesn't write for you.  It writes with you.  So this is a little thing.  And my eight-year-old is addicted to this.  And he's not a kid that really liked writing before.  But where, you know, you could say, I want to write a horror story.  And it says, ooh, a horror story.  How spine-tingling and thrilling.  Let's dive into the world of eerie shadows and chilling mysteries.

And this is an activity where the student will write two sentences. And then the AI will write two sentences.  And so they collaborate together on a story.  The students write, Beatrice was a misunderstood ghost.  She wanted to make friends but kept scaring them by accident.  And the AI says, poor Beatrice, a lonely spirit yearning for companionship.  One day she stumbled upon an old abandoned mansion, et cetera, et cetera.  I encourage you all to, you know, hopefully one day try this.  This is surprisingly fun.  Now to even more directly hit this use case.  And what I'm about to show you, everything I showed you so far is actually already part of Calmigo.  What I'm about to show you, we haven't shown to anyone yet.

# Enhancing Reading Comprehension and Writing Skills with AI

This is a prototype. We hope to be able to launch it in the next few months.  But this is to directly use AI, use generative AI, to not undermine English and language arts but to actually enhance it in ways that we couldn't have even conceived of even a year ago.  This is reading comprehension.  This is the students reading Steve Jobs' famous speech at Stanford.  And then as they get to certain points, they can click on that little question.  And the AI will then, Socratically, almost like an oral exam, ask the student about things.  And the AI can highlight parts of the passage.  Why did the author use that word? What was their intent? Does it back up their argument? They can start to do stuff that, once again, we never had the capability to give everyone a tutor, everyone a writing coach, to actually dig into reading at this level.  And you could go on the other side of it.  We have a whole workflow that helps them write, helps them be a writing coach, draw an outline.  But once a student actually constructs a draft, and this is where they're constructing a draft, they can ask for feedback, once again, as you would expect from a good writing coach.  In this case, the student we'll say, let's say, does my evidence support my claim? And then the AI not only is able to give feedback, but it's able to highlight certain parts of the passage and says, you know, on this passage, this doesn't quite support your claim, but once again, Socratically says, can you tell us why? So it's pulling the student, it's making them a better writer, giving them far more feedback than they've ever been able to actually get before, and we think this is going to dramatically accelerate writing, not hurt it.

§§§

# Personalized Education for Teachers
Now, everything I've talked about so far is for the student, but we think this could be equally as powerful for the teacher to drive more personalized education and, frankly, save time and energy for themselves and for their students. So this is an American history exercise on Khan Academy.  It's a question about the Spanish-American war.  And at first, it's in student mode, and if you say, tell me the answer, it's not going to tell the answer, it's going to go into tutoring mode.  But that little toggle which teachers have access to, they can turn student mode off, and then it goes into teacher mode.  And what this does is, it turns into, you could view it as a teacher's guide on steroids.  Not only can it explain the answer, it can explain how you might want to teach it.

# Benefits for Teachers
It can help prepare the teacher for that material. It can help them create lesson plans, as you can see doing right there.  It'll eventually help them create progress reports, it'll help them eventually grade.  So once again, teachers spend about half their time with this type of activity, lesson planning, all of that energy can go back to them or go back to human interactions with their actual students.  So, you know, one point I want to make, these large language models are so powerful, there's a temptation to say, well, all these people are just going to slap them onto their websites, and it kind of turns the applications themselves into commodities.  And what I've got to tell you is, I kind of thought that's one of the reasons why I didn't sleep for two weeks when I first had access to GPT-4 back in August.

# Enhancing AI Tutoring
But we quickly realized that to actually make it magical, I think it's really important to make it magical, but we quickly realized that to actually make it magical, I think what you saw with Conmigo a little bit, it didn't interact with you the way that you see chat GPT interacting, it was a little bit more magical, it was more Socratic, it was clearly much better at math than what most people are used to thinking. And the reason is there was a lot of work behind the scenes to make that happen.  And I could go through the whole list of everything we've been working on, many, many people, for over six, seven months, to make it feel magical, but perhaps the most intellectually interesting one is we realized, and this was an idea from an open AI researcher, that we could dramatically improve its ability in math and its ability in tutoring if we allowed the AI to think before it speaks.

# The Future of AI and Education
So if you're tutoring someone and you immediately just start talking before you assess their math, you might not get it right. But if you construct thoughts for yourself, and what you see on the right there is an actual AI thought, something that it generates for itself but it does not share with the student, then its accuracy went up dramatically and its ability to be a world-class tutor went up dramatically.  And you can see it's talking to itself here.  It says, the student got a different answer than I did, but do not tell them they made a mistake.  Instead, ask them to explain how they got to that step.  So I'll just finish off.

# The Role of AI in Education and the Need for Positive Use Cases
Hopefully, what I've just shown you is just half of what we are working on, and we think this is just the very tip of the iceberg of where this can actually go. And I'm pretty convinced, which I wouldn't have been even a year ago, that we, together, have a chance of addressing the two-sigma problem and turning it into a two-sigma opportunity, dramatically accelerating education as we know it.  Now, just to take a step back at a meta-level, obviously, we heard a lot today, the debates on either side.  There's folks who take a more pessimistic view of AI.  They say, this is scary, there's all these dystopian scenarios.  We maybe want to slow down.  We want to pause.  On the other side, there are the more optimistic folks who say, well, we've gone through inflection points before.

We've gone through the Industrial Revolution. It was scary, but it all kind of worked out.  And what I'd argue right now is, I don't think this is like a flip of a coin or this is something where we'll just have to wait and see which way it turns out.  I think everyone here and beyond, we are active participants in this decision.  I'm pretty convinced that the first line of reasoning is actually almost a self-fulfilling prophecy, that if we act with fear and if we say, hey, we just got to stop doing this stuff, what's really going to happen is the rule followers might pause, might slow down, but the rule breakers, as Alexander mentioned, the totalitarian governments, the criminal organizations, they're only going to accelerate.

And that leads to what I am pretty convinced is the dystopian state, which is the good actors have worse AIs than the bad actors. But I'll also talk to the optimist a little bit.  I don't think that means that, oh yeah, then we should just relax and just hope for the best.  That might not happen either.  I think all of us together have to fight like hell to make sure that we put the guardrails, we put in, when the problems arise, reasonable regulations, but we fight like hell for the positive use cases.  Because very close to my heart, and obviously there's many potential positive use cases, perhaps the most powerful use case, and perhaps the most poetic use case, is if AI, artificial intelligence, can be used to enhance HI, human intelligence, human potential and human purpose.  Thank you.  Thank you.  Thank you.
'''

# Gradio App

In [14]:
def run_gradio_app():
  import gradio as gr

  # Theming
  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )
  # Styling: Change max width
  css = """
    .gradio-container {max-width: 800px!important}
    .vspacer1 {margin-top: 50px}
  """

  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# ChatGPTLab 2.0", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")

      project_name = gr.Textbox(label="Project name")

      #
      # 1. Input Text
      #
      with gr.Tab("Input Text "):
        gr.Markdown("Please enter text")

        # Input text via UI
        gr.Markdown("### Input your text:")
        text_input = gr.Textbox(label="Enter text", placeholder="Your text here...", lines=10)
        text_output = gr.Textbox(label="Result")

        def text_save(text, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          f= open(proj_name + '/' +  folders['text-input'] + '/input_text.txt','w+')
          f.write(text)
          f.close()
          log_text = "Text saved."
          return log_text
        text_button = gr.Button("Save text")
        text_button.click(text_save, [text_input, project_name], text_output)

        gr.Markdown("")
        gr.Markdown("")

        # Input text via upload
        gr.Markdown("### Or upload your text:")
        upload_button = gr.UploadButton("Click to Upload a File", file_types=[".txt",".md"], file_count="single")
        file_output = gr.Textbox(label="Result")

        def upload_file(my_file, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          # copy to project directory
          full_upload_path = my_file.name
          just_the_filename = os.path.basename(full_upload_path)
          full_text_path = "./" + proj_name + '/' + folders['text-input'] + '/' + just_the_filename
          shutil.copyfile(full_upload_path, full_text_path)
          # check if file is empty
          f= open(full_text_path,'r')
          if f.mode == 'r': contents =f.read()
          f.close()
          log_text = just_the_filename + "\n"
          if len(contents) == 0:
            log_text = log_text + "Error: Upload file lengt 0 bytes"
          else:
            log_text = log_text + "Upload successful"
          return log_text
        upload_button.upload(upload_file, [upload_button, project_name], file_output)

      #
      # 2. Download full project
      #
      with gr.Tab("Download"):
        gr.Markdown("Download full project as ZIP file.")
        download_button = gr.Button("Download project")
        download_output = gr.File()

        def download_do(proj_name):
          full_text_path = "./" + proj_name
          shutil.make_archive('archive', 'zip', full_text_path)
          result = "Downloading " + full_text_path
          return "archive.zip"
        download_button.click(download_do, project_name, download_output)

      #
      # 3. xxx
      #
      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")
        greet_btn = gr.Button("Start", scale=0)
        def greet(name):
          result = "HALLO " + name + "!!!"
          return result
        greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

  demo.launch(quiet=True, share=False, debug=debug_mode)

#run_gradio_app()

# Test GPT

In [15]:
tmp = split_text_by_separator(test_text1, '\n', '\n', 'fix-nothing', 200)
print(text_stats(test_text1))
print(len(tmp))

Number of words / paragraphs / §§§ segements: 197 / 1 / 1
1


In [16]:
# test promptlib and templates

import textwrap

prompt_obj = get_prompt_from_lib(promptlib_global, 'summary-bullets', '1')
# prompt_obj = get_prompt_from_lib(promptlib_global, 'summary-bullets', '1')
print(json.dumps(prompt_obj, sort_keys=True, indent=2))

input_text = 'This is the input text. Lorem ipsum.'

prompt_txt_final1 = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-dyn', 0.5, 'toc')
prompt_txt_final2 = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-fix', 0, '')
prompt_txt_final3 = build_prompt_from_template(input_text, prompt_obj, 'lang-de', '', 0, '')
print(textwrap.fill(prompt_txt_final1, 120))

#write_prompt_to_lib(prompt_obj)

#print(json.dumps(promptlib, sort_keys=True, indent=2))

# myprompt = "The given text is delimited by triple backticks. Summarize the current text to succint and clear bullet points of its contents.
#          The length of the summary must be 200 words maximum."
# myprompt = "The given text is delimited by triple backticks.
#           Summarize the current text to a maximum of 15 succint and clear bullet points of its contents."
# myprompt = myprompt + "The maximum number of words should be 300 words in total. "
# myprompt = myprompt + "Write everything in German language. " # this is optional !
# myprompt = myprompt + "```" + input_text + "```"

{
  "category": "summarize",
  "description": "Max word length: 1100 for EN, 900 for DE. approx. 1 min processing time Example: Full book: 113 parts \u00e1 1000 words takes 15 min.",
  "id": "summary-bullets",
  "input-text-max": 500,
  "lang-de": "Write everything in German language.",
  "lang-en": "Write everything in English language.",
  "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
  "length-max-dyn": "The length of the summary must be {{length_calc(\"{input_text}\", {max_len})}} words maximum.",
  "length-max-fix": "The length of the summary must be 300 words maximum.",
  "note": "summarize to bullets v3",
  "prompt": "The given text is delimited by triple backticks. Write a detailed bullet-point summary. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
  "version": "3"
}
The given text is delimited by triple backticks. Write a detailed bullet-point summary. Write everything in German
lan

In [17]:
info_1.layout.visibility = "hidden"
info_1.close()
del info_1
button_1.layout.visibility = "visible"
button_2.layout.visibility = "visible"
button_3.layout.visibility = "visible"
button_4.layout.visibility = "visible"
#text_1.value="yoyo"

# Workbench

In [18]:
proj_name = 'S4HANA'
#create_file_directory(proj_name, True) # For dev: Initial folder structure
#create_file_directory(proj_name + '/' + folders['text-output'], True)
#create_file_directory(proj_name + '/' + folders['text-output-logs'], True)
#create_test_project(proj_name, '')

# Step 1
def step_1():
  model = 'gpt-3.5-turbo'
  prompt_name = 'summary-shortened'
  prompt_version = '1'
  prompt_comment = 'TODO: HIER EINE NOTIZ. HIER EINE NOTIZ. HIER EINE NOTIZ. HIER EINE NOTIZ.'
  prompt_obj = get_prompt_from_lib(promptlib_global, prompt_name, model, prompt_version)
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_file = execute_prompt({
      'model': model,
      'prompt-name': prompt_name,
      'prompt-version': prompt_version,
      'prompt-comment': prompt_comment,
      'loglevel': 0,
      'stop-after-step': 1,
      'proj-name': proj_name,
      'input-filepath': '', #'text-output',
      'input-filename': '', #'paraphrased.txt',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-de',
      'length-type': 'length-max-dyn',
      'length-max': 0.5,
      'toc-text': '',
      'result-filename': '01-paraphr-short.md'
  })
  print(result_file)
  a, b = result_file
  print(a)
  print(b)

# Step 2
def step_2():
  model = 'gpt-3.5-turbo'
  prompt_name = 'create-subheadlines'
  prompt_version = '1'
  prompt_obj = get_prompt_from_lib(promptlib_work, prompt_name, model, prompt_version)
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_file = execute_prompt({
      'model': model,
      'prompt-name': prompt_name,
      'prompt-version': prompt_version,
      'loglevel': 0,
      'stop-after-step': 3,
      'proj-name': proj_name,
      'input-filepath': 'text-output',
      'input-filename': '01-paraphr-short_gpt-3.md',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-same',
      'length-type': '',
      'length-max': 0,
      'toc-text': '',
      'result-filename': '01-paraphr-short-headlines.md'
  })

# Step 3
def step_3():
  model = 'gpt-3.5-turbo'
  prompt_name = 'summary-shortened'
  prompt_version = '1'
  prompt_obj = get_prompt_from_lib(promptlib_work, prompt_name, model, prompt_version)
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_file = execute_prompt({
      'model': model,
      'prompt-name': prompt_name,
      'prompt-version': prompt_version,
      'loglevel': 0,
      'stop-after-step': 3,
      'proj-name': proj_name,
      'input-filepath': 'text-output',
      'input-filename': '01-paraphr-short-headlines_gpt-3.md',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-de',
      'length-type': 'length-max-dyn',
      'length-max': 0.5,
      'toc-text': '',
      'result-filename': '02-paraphr-short.md'
  })

#step_1()
#step_2()
#step_3()

# Idea: chain by returning filename of last file => can be input for next

In [19]:
notiz = {
  "prompt": "I want you to act as a very proficient SEO and high-end. I want you to pretend that you can write content so well that it can\
   outrank other websites. All output must be 100% human writing style and fix grammar issues and change to active voice. Your task is to\
   write an article starting with SEO Title with a bold letter and rewrite the content and include subheadings using related keywords.\
   Write in the style and tone of a professional IT magazine for software experts. Write every sentence in your very own words and do not\
   only rephrase the given content. The article must be 100% unique and remove plagiarism. Do not write a summary or conclusions in the\
  last paragraph. Avoid lists. Include subheadings. {language_text} {length_text} Text:```{input_text}```"
}

In [20]:
proj_name = 'Handelsblatt'
#create_file_directory(proj_name + '/' + folders['text-output'], True)
#create_file_directory(proj_name + '/' + folders['text-output-logs'], True)
#create_test_project(proj_name, '')


# Step 1
def step_1():
  model = 'gpt-4'
  prompt_obj = get_prompt_from_lib(promptlib_work, 'summary-multiple-articles', model, '1')
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_text = execute_prompt({
      'model': model,
      'loglevel': 0,
      'stop-after-step': 0,
      'proj-name': proj_name,
      'input-filepath': '', #'text-output',
      'input-filename': '', #'paraphrased.txt',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-de',
      'length-type': '',
      'length-max': 0,
      'toc-text': '',
      'result-filename': '01-paraphr-short.md'
  })

#step_1()

# Current Work / Test

In [21]:
promptlib_global['summary-shortened-same-tone'] = {
      'description': 'Abgeleitet aus "summary-shortened"',
      'category': 'summarize',
      '1': {
          'note': 'Summary without changing the tone',
          "prompt": "Paraphrase the following text and shorten each paragraph of the paraphrased text to maxiumum the half of its original length by focussing\
            on the key aspects. Write in the same style and tone as the given text. Be detailed and precise\
            and use every content detail of the text. {language_text} {length_text} Text:```{input_text}```",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    }
text_from_file = "<File:>"
comment = 'Kommentar zum Projekt...'
# execute_project('S4HANA', 'gpt-4', promptlib_global, 'summary-shortened-same-tone', '1', 'length-max-dyn', 0, '', True, folders, text_from_file, comment)

In [22]:
comment = 'Kommentar zum Projekt...'
#execute_project('Handelsblatt Fr', 'gpt-4', promptlib_global, 'summary-multiple-articles', '1', 'length-max-dyn', 5, '' ,True, folders, '<File:>', comment)

In [23]:
promptlib_global['article-create-toc'] = {
      'description': '',
      'category': 'create-article',
      '1': {
          'note': 'Step 1: Create TOC',
          "prompt": "Here is some context for an article. The context is marked with 'Text:' and delimited by triple backticks. \
            Create a table of contents for an article titled 'Der Wert von Geschriebenem in Zeiten von Chat-GPT'. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    }
promptlib_global['article-create-chapter'] = {
      'description': '',
      'category': 'create-article',
      '1': {
          'note': 'Step 2: Create first chapter',
          "prompt": "Here is some context for an article titled 'Der Wert von Geschriebenem in Zeiten von Chat-GPT'. The context is marked with 'Text:' and delimited by triple backticks. \
            The table of contents is marked with 'TOC:' and delimited by triple backticks.\
            Write chapter 1 with a length of 500 words. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    }
text_from_file = "<File:>"

# Step 1
comment = 'Kommentar zum Projekt...'
#execute_project('article-future-of-text', 'gpt-4', promptlib_global, 'article-create-toc', '1', '', 0, '', True, folders, text_from_file, comment)

# Step 2
toc_text = f"""TOC: \`\`\`Inhaltsverzeichnis:
Titel: Der Wert von Geschriebenem in Zeiten von Chat-GPT
1. Künstliche Intelligenz und ihre Auswirkungen auf die täglichen Aufgaben
2. Die Rolle von Bots in der schriftlichen Kommunikation
3. Verhandlungen und Diskussionen in einer von Bots dominierten Welt
4. Die Auswirkungen von Künstlicher Intelligenz auf die Kreativität und die Wahrnehmung von Originalität
5. Die Auswirkungen von Künstlicher Intelligenz auf die Arbeit von Autoren und die Bedeutung von Charisma
6. Fazit: Die Zukunft von Geschriebenem in Zeiten von Chat-GPT.
\`\`\`"""
#execute_project('article-future-of-text', 'gpt-4', promptlib_global, 'article-create-chapter', '1', '', 0, toc_text, False, folders, text_from_file, comment)




In [24]:
#import_prompt_lib()
#import_helper_lib()
#print(test_pypandoc())

# Ideas
- Markdown Konvertierung wenn docx
- Chaining (ausgabefile)
- Artikel schreiben TOC, danach Kapitel


## How to Overcome GPT Token Limit: With Vector indices
- Vector search enables the inclusion of only pertinent information from the loaded data into the model's context.
- Steps to implement this solution include registering for a database service, splitting documents into smaller chunks, converting them into vectors, and querying the index to find relevant information.
- The provided code examples utilize the Pinecone database and OpenAI's Ada model for vector conversion, but other alternatives can be explored.
- https://uxplanet.org/how-to-overcome-gpt-token-limit-721c30a18d55

# Quick Action

In [25]:
#import_promptlib() # during dev: update promptlib from promptlib.ipynb
my_input_text = widgets.Textarea(placeholder='Enter input text', layout=Layout(width='50%', height='150px'), disabled=False)
button = widgets.Button(description="Summarize!")
def on_button_clicked(b):
  comment = 'Kommentar zum Projekt...'
  #execute_project('Handelsblatt', 'gpt-3.5-turbo', '', promptlib_global, 'summary-multiple-articles', '1', 'length-max-dyn', '', 0 ,True, folders, my_input_text.value, comment)
  execute_project('quick-action', 'gpt-3.5-turbo', 'Shorten this text, answer in German: ', {}, 'Manual prompt', '', '1000', '', 0, True, folders, my_input_text.value, comment)
button.on_click(on_button_clicked)
display(my_input_text, button)

Textarea(value='', layout=Layout(height='150px', width='50%'), placeholder='Enter input text')

Button(description='Summarize!', style=ButtonStyle())

# Manual Prompt

- Variable `input_dev` enthält Kontext
- Variable `prompt_dev` enthält Prompt

**Aufruf:**
```
execute_project('quick-action', 'gpt-3.5-turbo', prompt_dev, {}, 'Manual prompt', '', '500', '', 2, True, folders, input_dev, comment)
```
- quick-action: Projekt/Ordnername
- '500' Anzahl Wort-Split bei langen Texten
- 2 => Abbruch nach Step 2 (für Dev), sonst 0


In [108]:
# @title
input_dev = f"""
 Herzlich willkommen zum Content-Performance-Podcast mit Fabian Deckert und Benjamin Odani, der Online-Marketing-Podcast für Fortgeschrittene. Hallo Fabian. Hallo Benjamin. Und hallo Michael. Ja, hallo zusammen. Heute sprechen wir darüber, wie Deloitte Thought Leadership Content entwickelt und dafür haben wir uns einen super Gast geholt in unserem Podcast, und zwar den Michael Gramp. Du bist Chefökonom bei Deloitte in der Schweiz, Leiter des Research Teams und eben auch verantwortlich für diese Thought Leadership Publikation. Vielen Dank, dass du dir die Zeit nimmst, um mit uns darüber zu sprechen. Ja, unbedingt. Und vielen Dank für die Einladung und toll, dass ich in einem meiner Lieblings-Podcasts, die ich ja schon seit Jahren kenne, zu Gast sein darf. Ja, also vollkommen zurecht, finde ich, und vielen Dank für das Lob und dann lass uns doch mal direkt reinspringen in das Thema, also das, weil Thought Leadership ist ja sowas, das wird viel diskutiert, aber die Wenigsten machen das ja wirklich, weil es ja auch teilweise sehr intensiv ist, was die Umsetzung angeht und deswegen direkt die erste Frage, warum macht ihr denn oder warum investiert ihr denn als Deloitte in Thought Leadership Content? Ja, also das ist eine gute Frage und was man vielleicht vorausschicken muss, bei uns ist es keine Auftragsarbeit, denn häufig sehe ich Content-Inhalte, die von Firmen erstellt werden, die aber teilweise irgendwelche Auftraggeber im Hintergrund haben, extern, also das ist mal wichtig, das machen wir nicht. Es ist wirklich ein reines Investment, ein Geldinvestment letzten Endes auch, weil du natürlich ein Team dazu brauchst und Thought Leadership hilft dir dabei, sage ich mal, ein Unternehmen, aber auch eine Einzelperson zu positionieren als Vordenker, als innovativer Thought Leader zu einzelnen Themen als Experte und diese Positionierung hilft natürlich bei der, sage ich mal, Marke, beim Auftritt des Unternehmens etc. und dadurch langfristig kannst du natürlich Kundenbeziehungen aufbauen oder eine Kundenbindung verstärken dadurch, also das ist natürlich das langfristige Ziel immer dabei. Ja, sprechen wir später nochmal über KPIs, sprechen wir später nochmal, ne? Ja, genau, KPIs ist natürlich eine ganz wichtige Sache dabei und die können wir ganz später noch im Detail besprechen, ja. Jetzt habt ihr, die Leute ist ja richtig groß, also ihr habt, ich glaube, 450.000 Angestellte weltweit, so weit ich weiß. Richtig, richtig. Und da fragt man sich natürlich, wie ist das denn dann organisiert? Also wie habt ihr diesen, ihr sagt, wir wollen strategisch investieren in Thought Leadership und in Thought Leadership Content, um uns eben im Markt zu positionieren als Vordenker. Wie habt ihr das dann in so einem großen Konzern organisiert? Genau, also wichtig ist mal zu wissen, die Leute, also wir sind ja die größte Beratungsgesellschaft der Welt und wir sind einzelne Ländergesellschaften, also wir sind nicht der klassische Konzern, wie es jetzt vielleicht ein Google wäre oder ein Mercedes oder so, die wo ein großes Headquart ist und alles bestimmt, sondern wir sind regional aufgestellt und daher ist auch der Thought Leadership-Bereich auf drei Ebenen gegliedert. Es gibt globales Thought Leadership, also ein Team, das sitzt in den USA bei uns, das globale Themen quasi für alle Ländergesellschaften erstellt. Logischerweise ist das ein bisschen mehr high-level, also ein bisschen oberflächlicher. Man kann natürlich jetzt nicht was schweizspezifisches oder spezifisches für Deutschland darin erstellen, aber man hat eine gute Pipeline, eine Inhaltspipeline und es werden die für uns wichtigen Themen global erstellt. Dann haben wir regionale Research-Teams und Content-Teams, die Thought Leadership erstellen. EMEA, die sitzt bei uns in Großbritannien und in Deutschland. Es ist ein Team, das europäische Themen, aber auch mal afrikanische Themen, also EMEA ist ja Afrika und Far East as well, mit aufnimmt und Mittel-East, sorry, nicht Far East. Und dann gibt es die lokalen nationalen Teams, hat nicht jedes Land. Wir sind in 160 Ländern weltweit tätig. Das haben eher ein bisschen die größeren Gesellschaften Großbritannien, Deutschland, die Schweiz und haben ja lokale Teams, wie zum Beispiel in der Schweiz. Und wir bearbeiten ausschließlich Schweizer Thought Leadership Themen oder die deutschen Kollegen eben eher die deutschen Themen und haben dann natürlich auch andere Schwerpunkte als in Deutschland. Ist die Automobilindustrie nach wie vor wichtig, weshalb hier mehr Content für die Automobilindustrie erstellt wird. In der Schweiz weniger, weil es für uns nicht so ein wichtiger Sektor ist. Dafür sind es dann andere Themen. Kannst du mal so ein paar Beispiele nennen, was ihr so für Themen macht? Ja, genau. Also es gibt so industriespezifische. Das hängt wirklich von der Branche ab. Also hier in der Schweiz ist natürlich die Bankenbranche wichtig, die Versicherungsbranche, Life Science, die MEM-Industrie, wie wir die hier nennen. Das ist also Maschinenbau etc. In Deutschland ist dann mehr ein bisschen Automobilindustrie, aber auch noch andere Pharma- und chemische Industrie. Und dann gibt es industrieübergreifende Themen. Das sind natürlich sehr häufig Technologiethemen. Das sind aber auch Themen wie die Zukunft der Arbeit, das heißt, wie wir in Zukunft arbeiten werden. Das sind Themen wie Cyberkriminalität, also Cybersicherheit, wie wir das nennen. Das sind dann die industrieübergreifenden Themen. Also wir greifen immer Themen auf, die für Unternehmen wichtig sind und die gerade so diese Hot Topics, diese aktuellen Themen sind oder die in Zukunft kommen. Denn das sind Herausforderungen, wo wir dann glauben, dass wir Lösungen haben für Unternehmen. Und diese Sachen thematisieren wir dann. Und wie ist das denn? Also ist das wirklich, seid ihr Thought Leadership? Ist das so das, was über eurem Team drübersteht? Oder ist das eher so noch im Marketing aufgehangen und dann macht man das noch so nebenbei? Oder ist es wirklich eure Kernaufgabe, an Thought Leadership zu arbeiten? Kein organisatorisch oder von Titel her? Also wir sind nicht im Marketing aufgehängt und wir können gerne nachher noch mal drüber sprechen, wie man es eigentlich nicht machen sollte, weil den größten Fehler, den die meisten Firmen machen, sie hängen es im Marketing auf. Können wir aber später noch dazu sprechen. Können wir gerne auch jetzt machen. Warum? Lass uns erst mal kurz. Also wir sind ganz klar Content-Driven, das heißt, Inhalt treibt alles andere bei uns. Und die Erstellung von Inhalt Thought Leadership Inhalt steht ganz am Anfang. Den produzieren wir und damit füttern wir Kommunikation, Marketing, Kundenbeziehungen etc. Und deswegen sollte auch ein Research Team, das diese Sachen arbeitet, nicht im Marketing sitzen. Jetzt kann man ein bisschen drüber streiten, rein von der Organisation, wenn das Team natürlich relativ unabhängig arbeiten kann und die Experten hat, kann das natürlich organisatorisch schon im Marketing sitzen. Was ich aber häufig sehe, ist, dass Marketing oft glaubt, sie können Thought Leadership Content produzieren. Und das ist in der Regel nicht der Fall, weil du brauchst wirklich die Contentexperten dafür. Und das sind meistens Researcher, es sind Leute, die gut schreiben können. Noch am ehesten funktioniert es in der Kommunikationsabteilung. Aber noch besser ist es, einen losgelösten Bereich bis in eine Stabsstelle zu haben, so wie wir das haben. Also ich bin, unser Bereich ist separat und auf gleicher Ebene gibt es dann Kommunikation und Marketing. Aber was ich immer sage, Content drives Marketing. Inhalte treiben Marketing und nicht andersherum. Und das Problem ist, wenn Marketing die Inhalte erstellt, glauben sie häufig, Marketing treibt Inhalte und das ist eben falsch. Das hat man vor 10, 15 Jahren noch gemacht. Und wenn du dir die Inhalte, die Firmen vor 10, 15 Jahren oder teilweise auch noch heute erstellen, dann siehst du, dass das eher Marketing für Chaos Messages waren. Das funktioniert heutzutage nicht mehr. Es machen aber immer noch viele falsch. Also diese klassische nur Marketing-Message rauszuhauen, das funktioniert nicht mehr. Also musst du einen anderen Weg gehen und es geht über Inhalte, die relativ neutral sind und nicht primär im ersten Schritt dem Verkauf dienen. Kannst du das mal genau so ein bisschen an einem Beispiel erklären? Also wenn du, was bedeutet denn dann Research für euch? Also wo macht ihr dann eine eigene Studie und geht darüber dann raus? Oder kannst du das mal vielleicht... Erzähl mal, an was in der Studie hast du letzte Woche gearbeitet oder letzten Monat? Genau. Immer ein gutes Beispiel, Generative AI. Ein heißes Thema, derzeit wird in Unternehmen sehr intensiv diskutiert. Keiner hat noch so wirklich eine Ahnung, was da auf uns zurollt. Wir haben sehr schnell reagiert, weil wir gemerkt haben, da ist eine Nachfrage da. Und ein bisschen so ein White Space, wie wir das nennen. Also wir schauen auch immer nach Themen, die noch nicht von 10 anderen behandelt werden, weil wir wollen ja ein bisschen Aufmerksamkeit auch. Das ist dann die Medienarbeit damit erzeugen und auch mit was Neuem rauskommen. Klassisches Beispiel, wir haben eine Befragung, Konsumentenbefragung, dann zu dem Thema Generative AI auf dem Arbeitsplatz durchgeführt. Die Ergebnisse ausgewertet und jetzt erstellen wir einen Bericht drumherum. Das wird letzten Endes dann schon eine Studie sein, aber jetzt auch hier wichtig, was man heutzutage auch nicht mehr macht, aber immer noch viele machen. Die erstellen eine Studie dann als PDF auf die Website fertig. Also das geht natürlich überhaupt nicht mehr, funktioniert nicht mehr. Und dann wundern sich die Leute, dass die KPI so schlecht sind. Wir machen immer einen Digital First Approach, nennen wir das. Wir schauen erst mal an, wie soll eigentlich das Ergebnis auf unseren digitalen Plattformen aussehen? Weil dort werden die Sachen eben inzwischen am meisten gelesen. PDF laden sich manche noch herunter, aber Digital First, das heißt Website, Microsite auf dem Mobiltelefon. Und dann erstellen wir die Inhalte, eine Studie, die dann schon eine finale Studie ist. im PDF-Format haben, aber nicht im Mittelpunkt stehen. Und was jetzt auch noch wichtig ist, ist jetzt nur ein Beispiel. Wir nehmen eine Befragung. Befragungen sind immer schön, weil dann hast du eigene Daten. Wir machen natürlich auch Desktop-Research. Das heißt, wir suchen natürlich auch andere Erkenntnisse in dem Bereich raus. Und dann arbeiten wir auch sehr viel mit Interviews. Und zwar mit Interviews mit Externen auch und holen die Expertenmeinung aus unterschiedlicher Seite rein. Und auch noch ganz wichtig, wir haben zwar Research Team, aber wir sind natürlich auch nicht bei allen Themen die Experten. Wir arbeiten hier sehr eng mit den anderen Kollegen in unseren anderen Abteilungen zusammen, die beim Kunden genau diese Themen behandeln. Weil die wissen ganz genau, wo drückt der Schuh besonders, was sind Lösungsansätze etc. Und dieses Wissen bringen wir mit rein, aber auch zusammen mit den Externen. Weil was wir nicht wollen, ist, dass es so eine reine Deloitte-Studie wird, wo nur unsere Meinung abgebildet wird. Auch das funktioniert heutzutage nicht mehr. Man muss diese Plattformen, die man hier betreibt, öffnen, um auch Externen die Möglichkeit geben, sich zu positionieren und die Meinung rauszuholen. Und da sind wir jetzt dabei. Bevor Benjamin jetzt reingeht, ich weiß, der ist schon in den Schlagzeugheim. Ich würde gerne eine Zwischenfrage noch stellen, bevor die ein bisschen untergeht, weil du hast gesagt, Distribution. Also man macht was, als PDF hast du gesagt. Klar, das ist sozusagen der zentrale Content oder das, was dann dabei rauskommt. Dann hast du gesagt, ihr stellt es auf die Webseite und auf eine Microsite hast du gesagt. Also das heißt, ihr baut für jede Studie auch noch eine einzelne Unterseite? Für die Großen, für die Großen. Also für die Größeren und für jeden Zuhörer einfach mal auf unsere Schweizer Seite gehen. Ein gutes Beispiel ist unsere jährliche Uhrenstudie. Das ist eine klassische Microsite, wo eine separate Seite aufgebaut wird, wo wir natürlich auch viel freier sind bezüglich grafischer Elemente. Da arbeiten wir auch mit einer Agentur dann zusammen, Interaktivität und so weiter. Und es sieht einfach cooler aus. Und wir merken einfach, mit Microsites bleiben die Leute zwei bis dreimal länger auf der Seite, wie auf der Standard-Website, weil ich eben ganz andere Sachen mit einbauen kann. Also richtig, eine separate Microsite. Die verlinken wir auf jeden Fall in den Shownotes, damit man sich das auch noch angucken kann. Über Microsites diskutieren wir heute nicht. Aber es ist sozusagen auf einer hängenden Unterseite. Also es ist ja sehr aufwendig, das zu machen. Da sieht man auch, wie wichtig euch das dann ist. Und aber macht ihr denn auch dann, wie, ich meine, gut, dann hat man das jetzt auf der Seite, hat eine Microsite, aber wie tragt ihr das denn dann nach außen? Brecht ihr dann Sachen raus und streut die dann über eure Social-Media-Kanäle? Genau, richtig. Kannst du vielleicht einmal noch gerade erzählen, wie dann nachher das Seeding sozusagen weitergeht? Genau, weil das ist ja das Entscheidende immer. Was nützt es, den besten Content zu haben, aber nicht zu wissen, wie ich ihn verteile? Also erstmal wichtig, ich bin ein großer Fan von Gary Vaynerchuk's Content-Pyramide. Das heißt, wir ermöglichen großen Basis-Content. Und um diese Shelf-Life, also damit ich länger die Kampagne fahren kann, nutzen zu können, hole ich dann aus diesem zum Beispiel eine breite Studie, die dann natürlich abgebildet ist auf einer Microsite, hole ich einzelne Inhaltsstücke raus und mache neuen Inhalt daraus. Das ist so, wie wenn ihr jetzt einen Podcast aufnehmt, aber dann noch 5 Social-Media-Posts macht und hier noch ein kurzes Video etc. Also Content-Pyramide, Einzelteile brechen wir raus. Unsere Hauptkommunikationskanäle sind LinkedIn und Twitter. Das sind im Social-Media-Bereich unsere Hauptkanäle. Wir haben bei den Leuten natürlich auch Instagram usw., aber das ist eher dann für unsere HR-Abteilung, für Talent usw. Und mit TikTok machen wir auch ein bisschen was, aber 4 für unseren Content. LinkedIn, weil wir dort am besten unser Zielpublikum erreichen. Dann nimmt natürlich Marketing auch das für Direct-Mailing. Newsletter machen wir jetzt in dem Sinn nicht mehr so häufig, aber wir machen natürlich auch viele Webinars für Events usw. Und da gibt es überall die Möglichkeiten, das zu streuen. Und dann natürlich die Medienarbeit. Da kommt jetzt unser Kommunikationsteam rein. Interne Kommunikation ist wichtig, dass also unsere internen Leute, jetzt in der Schweiz sind wir bei 2.500, davon informiert sind, weil die teilen es dann auch wieder weiter. Und natürlich die Medienarbeit, also versuchen, die Inhalte zu platzieren bei Print, Online, TV und Radio. Das sind eigentlich die 4 Großen. Das gelingt uns mal gut, mal weniger gut. Das kann man immer nicht so ganz steuern, weil Medien brauchen ein bisschen Glück, dass die gerade an dem Tag oder in der Woche Interesse an dem Thema haben. Und vielleicht noch zu LinkedIn. Da arbeiten wir auch mit Live Social. Das ist so ein Tool. Es gibt ähnliche Tools, wo unser Social Media Team eigentlich zum Beispiel zu einer Studie, zu einer neuen Kampagne Posts vorbereitet. Und jeder Mitarbeiter hat Zugriff auf diese Plattform, die verknüpft ist mit LinkedIn und kann dadurch relativ schnell eigene Posts kreieren. Weil das Hauptproblem, ihr wisst, einen LinkedIn Post zu machen, gut, das dauert natürlich. Das mache ich nicht in zwei Minuten. Aber wir helfen da unseren Mitarbeitern bis auf dieses Tool. Du gibst ein Future of Health, Zukunft der Arbeit ist ein spannendes Thema und da siehst du fünf Posts dazu. Nicht nur eigene Sachen, auch mal ein Artikel und so weiter. Oh ja, das würde ich gern teilen über mein LinkedIn Netzwerk. Und da kannst du auch einstellen, an welchen Tagen und so weiter. Du kannst eigentlich, mache ich immer Montag, komplette Pipeline eine Woche schon mal vorausplanen, um wie viel Uhr, an welchem Tag, welche Posts mit reingehen. Und es geht dadurch natürlich schneller, weil ich nur noch kleine Anpassungen machen muss beim Text. Also es gibt unterschiedliche Möglichkeiten, wie wir das treiben. Und wie viele seid ihr in eurem Fortleadership Content Team, Schrägstrich Research Team? Du hast ja gesagt, wie eine Stabsstelle organisiert. Kannst du da mal sagen, wie viele seid ihr? Fünf. Fünf Personen. Deutschen Kollegen haben natürlich mehr, weil sie größer sind und mehr Themen bearbeiten. Deutschen Kollegen haben, glaube ich, so um die 20, zwischen 20 und 25 Leute. Genau. Und ihr seid eben, wie du, ich finde das sehr spannend, was du gesagt hast, dass ihr wirklich unabhängig eure Forschung macht, sozusagen. Eure relevanten Themen macht, die nicht zu marketinglastig sind. Also nicht, wo sozusagen keine Produktkommunikation stattfindet, keine werbliche Kommunikation, sondern wirklich fachliche Inhalte, die ihr dann übergebt ans Marketing, an PR, an die verschiedenen sozusagen Abteilungen, die für die Distribution zuständig sind und auch für die Reichweite dann. Korrekt. Und das ist ein ganz, ganz wichtiger Erfolgsfaktor. Die Leute lesen, also in Unternehmen. Unser Zielpublikum sind ja die Unternehmen C-Level, mittleres Management, die sind an diesem Marketing-Blabla nicht mehr interessiert. Vergiss es. Also das kommt einfach nicht an. Aber an Content, und da gibt es ja viele Untersuchungen dazu, an wirklich guten Inhalten sind die sehr interessiert. Und die sehen es dann auch an, wenn Sachen gelesen wird, Feedback und so weiter. Oder wenn ich für Interviews anfrage, ich bekomme fast immer einen Termin, weil ich einfach jemand als Experte anfrage zu einem Thema. Also er hat jetzt nicht Angst, ich will ihm eine Dienstleistung verkaufen. Und das ist ganz entscheidend. Das Gleiche beim Podcast. Ihr macht es ja genauso. Ich auch mit meinem Podcast, ich interviewe niemand von den Leuten, sondern ich interviewe Externe. Und dadurch wird es natürlich für viele noch interessanter. Das machst du ja schon lange, euren Future Talk Podcast. Darüber haben wir uns ja auch kennengelernt, schon vor vielen Jahren. Richtig, vor vielen Jahren, genau. Du hast ja zur ähnlichen Zeit natürlich unsere Podcasts gelauncht, richtig. Ja, finde ich sehr spannend. Da habe ich ja auch damals schon sehr gefreut und da höre ich immer wieder regelmäßig rein, weil wirklich konsequent Externe auch eben wirklich aus dem C-Level-Bereich auch durchaus eben selbst auch reinholt und interviewt. Ja, genau. Und wir bieten eigentlich, und das machen wir auch vor allen Dingen auf den Microsites, die vorhin angesprochen wurden, wir öffnen, wir haben vor Jahren schon diese Plattform geöffnet für Externe. Das machen auch viele Unternehmen, nicht? Da werden nur die eigenen Sachen publiziert. Wir holen auch Externe rein, zum Beispiel mit Interviewserien, auch schriftlich mal kurze Interviews und das hilft natürlich sehr bei der Kundenbindung. Also das ist wahrscheinlich einer der wichtigsten Sachen, warum wir auch Content kreieren, Kundenbindung entweder aufzubauen oder zu verfestigen, weil wir bieten Externen eine Plattform, sich selbst zu präsentieren als Experte und das mag ja jeder. Also ich habe jetzt schon gehört, aus Studien, Interviewserien, sozusagen Artikel, Fachartikel drumherum und das wiederum brecht dir ja sehr stark auch auf verschiedene Branchen runter. Ihr habt ja einen sehr starken auch Branchenansatz, das finde ich auch nochmal sehr spannend. Kannst du denn nochmal sagen, wer schreibt, also ihr schreibt dann auch aus dem Research-Team, schreibt dann auch die Artikel zu den einzelnen Industrien? Genau, richtig, richtig. Natürlich haben wir so innerhalb vom Team ein bisschen Expertise. Also wir haben jetzt einen, der halt früher auch bei einer Bank gearbeitet hat, der macht natürlich die Bankgeschichten, dann natürlich, sagen wir mal Public Sector, also öffentliche Verwaltung haben wir jemand aus dem Bereich Konsumgüterindustrie ist wichtig. Genau, also die schreiben das dann, aber immer in enger Abstimmung mit unseren, sage ich mal, Experten, die nahe beim Kunden sind, mit den Beratern und so weiter, damit wir da eben sicher sind, dass wir auch die richtigen Inhalte bringen. Aber so dann auch das Schreiben und vom Stil her, das ist ja auch noch sehr wichtig, das machen wir alles. Und was auch noch wichtig ist, darum sage ich mal, was wir machen, wir machen Forschung, wir machen aber keine akademische Forschung, würde ich mal sagen, sondern eher die angewandtsorientierte Forschung. Wir bringen Lösungsansätze und wir schreiben keine akademischen Papiere, weil die liest keiner, mal ganz ehrlich. Also letztendlich die Akademiker kritisieren, aber innerhalb des akademischen Zirkels ist es durchaus sinnvoll, diese academic papers, aber die liest ja keiner. Ich glaube, das war die Weltbank, wo man selbst veröffentlicht hat, durchschnittliche Downloads von ihren Studien, die oft 100, 150 Seiten sind. Das waren irgendwie im Schnitt nicht mehr als 20. Also das ist natürlich völlig lächerlich, muss man sich mal vorstellen. Und auch die Sprache ist daher sehr, sehr wichtig. Da kommt unser Job schon sehr nah ans Journalistische rein. Wir müssen so schreiben, dass es jeder versteht einfach. Und dass man es auch schnell lesen kann. Also unsere Studien sind auch wesentlich kürzer in den letzten Jahren geworden, weil die Leute, man kann es jetzt beklagen, aber es ist eben so, die haben nicht mehr die Zeit, vor allen Dingen in den Unternehmen. Es muss schnell gehen, da muss eine knackige Summary sein, da müssen die Visuals drin sein. Also wir haben viel mehr, arbeiten jetzt auch mit Bildern, mit Grafiken, mit Abbildungen als vor zehn Jahren. Und dadurch bleiben die Leute auch länger dran und lesen die Sachen auch. Jetzt muss ich zwei Sachen einmal nochmal nachfragen, weil wir da wieder so drüber gehüpft sind. Also einmal würde ich einfach eine ganz blöde Frage stellen. Und zwar, warum macht ihr Branchen? Also warum unterteilt ihr das in diese Branchen? Was ist eure Erfahrung, dass das sinnvoll ist, das so thematisch aufzuteilen? Das hat damit zu tun, weil wir bei Deloitte, genauso wie auch unsere Konkurrenz, eben eine Matrix-Organisation haben. Wir haben nach Industrien die Experten und nach Themen. Und daher, es gibt Themen, die sind industrieübergreifend wichtig, Zukunft der Arbeit, da kann ich mit jedem HR-Verantwortlichen, egal von welchem, sprechen. Aber es gibt eben auch industriespezifische Themen, zum Beispiel digitale Währung oder kontaktloses Zahlen. Das ist dann primär ein Thema für die Banken und vielleicht noch für die Konsumgüterindustrie. Also es hängt alles immer vom Zielpublikum ab. Also bei uns geht es ja immer erstmal, wer ist unser Zielkunde? Wir schreiben ja jetzt nicht einfach so los, ein spannendes Thema, sondern wir überlegen uns der erste Schritt immer, für wen schreiben wir das eigentlich? Wen adressieren wir? Und da bist du halt bei vielen Themen, hast du entweder Industriethemen oder die übergreifenden Themen. Und daher machen wir Industriestudien, aber auch die anderen Studien. Genau, das hätte ich eigentlich auch erwartet, dass ihr da auch strukturell vom Unternehmen her so aufgebaut seid. Und auf der anderen Seite geht es natürlich auch darum, die Leute abzuholen. Jetzt einfach ganz blöd gesagt, wenn ich auf der Homepage bin und ich möchte Informationen für meine Branche finden, ist das ja auch schon wieder eine Einordnung, wenn ich mich da auch wieder finde, dass ihr da halt auch die Experten für seid. Und das Zweite war, was mir so ein bisschen aufgefallen ist in dem, was du uns erzählt hast, ist, dass du auch schon Lösungen angesprochen hast. Also der Content ist nicht werblich in dem Sinne. Aber man möchte ja schon auch transportieren, dass man sehr viel Ahnung davon hat und natürlich auch Lösungsansätze bringt. Im Gegensatz zu den ganzen wissenschaftlichen Studien, die das ja wirklich sehr global mal betrachten und vielleicht auch einen dann, bevor man eine Lösung findet, zu Tode gelangweilt hat. Es geht ja auch darum, dann auch ein Ergebnis zu produzieren. Also von daher würde mich schon interessieren, ob ihr die Themen auch auf eure Leistungen ein Stück weit abstimmt, dass ihr dann auch sagt, vielleicht auch in Form von einer Case Study oder dass man sagt, also wie bringt man denn dann in den Content auch das Angebot rein, was man hat? Ist es wirklich so unwerblich oder wie würdest du sagen, wie ist da deine Erfahrung oder wie geht ihr das an, dass man dann aber im Endeffekt aber dann auch signalisiert, dass ihr ja auch diejenigen seid, die man nachher fragen kann, wenn man dieses Problem zum Beispiel hat mit der Kryptowährung oder wenn man das implementieren will oder wenn ich als Bank wissen will, was für ein Risiko besteht denn das für meine Organisation, wenn es jetzt Kryptowährungen gibt? Ja, jetzt mal als Beispiel. Wie kann man sich denn da als Beratungsunternehmen dann trotzdem positionieren? Eigentlich genauso, wie es du es beschrieben hast. Und ich nehme mal den aktuellen Beispiel, an dem ich arbeite. Es gibt dann eben im zweiten Teil von der Studie gibt es in dem Teil Risikovermeidung, also wo ich dann zum Beispiel die Risiken, wir haben analysiert, was sind die Risiken, was müsste ich tun, um die Risiken zu vermeiden oder was gibt es sonst noch für Empfehlungen, was Unternehmen tun müssen, um dies oder jenes zu tun. Aber wir schreiben das nicht in dem Stil, wir wissen es und so weiter, weil nicht so direkt. Aber unterschwellig, wenn du natürlich jetzt einmal noch zwei, drei Seiten über diese Lösungsansätze schreibst, ist ja unterschwellig eine Aufforderung für jemand A, die kennen sich aus und darum geht es ja zu signalisieren. Wir sind in Thordly, da sind wir wieder bei der ersten Frage. Die kennen sich aus, oh ja, die Lösungsansätze macht Sinn, könnte für mich auch interessant sein und über den zweiten, dritten, nächsten Schritt kontaktieren sie vielleicht uns. Ja, sehr, sehr spannend, dass du jetzt auch sagst, es kommt sozusagen automatisch über die… Ja, das muss das Ziel sein und nicht dieses Push. Ja, und jetzt lassen wir mal über die KPIs sprechen, die wir schon am Anfang schon so, haben kurz fallen lassen, oder du. Wie misst du denn dann den Erfolg? Also, was sind denn dann wirklich, ja, die Zahlen? Von Thordly und Content. Genau, richtig. Wir haben eigentlich vier Bereiche, wir haben digitale KPIs, wir haben klassische Marketing KPIs, wir haben Medien KPIs und wir haben Kunden KPIs. Am leichtesten sind natürlich die digitalen KPIs, das wisst ihr, weil die kann man halt schön messen. Ja, und das sind die klassischen, also brauche ich jetzt, glaube ich, nicht so ganz im Detail, sind halt die Website, Visits, Scroll Rate, Bounce Rate, Time on Page, Leads und so weiter. Dann haben wir Organic Social Media und da teilen wir Organic Social Media und Paid, also bezahlte Kampagnen. Wir machen auf LinkedIn auch bezahlte Kampagnen, das funktioniert ja da recht gut. Und da geht es dann natürlich auch um Impressions, Klicks, Likes, Shares, Total Engagement. Also, haben wir auch einige Tools, unser Digital-Team natürlich Tools, um das zu messen entsprechend. Ich sehe dann immer nur die Auswertung, also ich bin jetzt da bei der Analyse nicht so direkt involviert. Und dann haben wir bei den Marketing KPIs, das sind alle kundenbezogenen Zusammenhänge, also das heißt, wie viele Touchpoints, also Berührungspunkte man eigentlich durch eine Studie mit Kunden hat. Das kann schon bei der Erstellung der Kunden, beim Inhalt sein, also bei der Erstellung Inhalt komme ich, weil gleich noch bei den Kunden KPIs dazu. Das sehen wir oft bei Marketing auch viel mit Events oder irgendwelche Mittagessen, wo man dann mit Kunden zusammensitzt und dann natürlich das vielleicht auch als Aufhänger nimmt, vielleicht auch für ein Event. Also, viele Events machen wir, basieren auf Ergebnisse von der Studie zum Beispiel. Also, das sind dann auch KPIs. Dann Marketing haben wir noch so diese Qualified Leads, wie wir es ja nennen. Das heißt, gibt es dann wirklich irgendwelche Engagements über welche Kanäle auch immer und kommt dann im zweiten, dritten Schritt vielleicht doch mal ein Gespräch über ein Projekt zustande. Die sind aber am schwierigsten zu messen, weil wir das natürlich nicht immer mitbekommen. Also, wir können nicht alle Mitarbeiter kontrollieren. Der hat jetzt eine Studie erwähnt bei einem Kunden und er fand es so interessant, dass er ein zweites Meeting hatte über dieses Thema und zwei Schritte später ist ein Auftrag herausgekommen. Das ist schwierig zu messen. Dann haben wir die Medien KPIs, die sind auch relativ leicht zu messen. Also, die Anzahl der Clippings in Zeitungen, Online, TV, Radio. In dem Zusammenhang dann so genannten Organic Reach. Also, das heißt, die Leser, Hörer, Zuschauer, also die dahinterstecken. Ist ja natürlich ein Unterschied, ob ich jetzt in einem Artikel mit einer Studie bin, in einer Lokalzeitung oder in der FAZ. Das ist ein großer Unterschied. Oder in den Abendnachrichten oder eher bei einem lokalen Radiosender. Share of Voice messen wir. Also, das messen wir bei Medien, weil da geht es auch. Share of Voice. Social Media auch ein bisschen. Also, wir vergleichen uns mit unseren direkten Konkurrenten und sehen, wie viele Medien Echo Tee hatten. Und da teilen die natürlich positiv, negativ. Das ist natürlich ganz wichtig. Und auch digital schauen wir natürlich, was die anderen auf LinkedIn und so weiter machen. Und dann haben wir noch die Kunden KPIs. Die sind jetzt für mich, für mein Team am wichtigsten, weil Ziel ist natürlich, von diesem Thought Leadership möglichst viel Kunden in der Aktivität zu erzeugen. Das sind persönliche Kundengespräche. Und da kommen wir in die Interviews herein. Darum führe ich sehr gerne Interviews im Rahmen von Studien, weil wenn ich da mit fünf Head of HR Leuten von Unternehmen spreche, ist das natürlich schon mal ein Checkpoint. Weil ich meine, das ist FaceTime mit jemanden oder halt jetzt dann über Zoom oder so. Das zählt mit rein. Auch Follow-up. Wir nutzen natürlich die Studien dann auch für weitere Diskussionen bei Kunden. Dann gibt es noch die externen Experten oder auch Politik. Politik ist auch wichtig bei manchen Themen. Also, ich bin häufig bei, ist ja bei uns die Bundeshauptstadt, wo ich dann auch mit Leuten aus der Politik zu einzelnen Themen rede oder auch mit der Nationalbank. Genau. Und dann gibt es noch die interne Kommunikation und Interaktion. Das ist auch für unser Team wieder wichtig. Das heißt, wie gelingt es mir, dass ich halt mal beim Thema auch mal mit unserem CEO darüber spreche, weil ich dann mit ihm vielleicht gemeinsam meinen Medientermin wahrnehme und so weiter. Also, das ist dann noch das, was intern auch möglich ist. Das ist vielleicht die größte Herausforderung in einem großen Unternehmen. Mir ist schon häufig aufgefallen, Externe kennen unsere Studien oft besser als die internen. Und da ist es interne immer wichtig, die Leute abzuholen. Hey, hier haben wir was Gutes. Und das reicht halt nicht nur aufs Intranet zu stellen oder mal ein Newsletter, den keiner mehr liest, sondern da muss ich oft die Leute ein bisschen persönlich abholen. Ich glaube, wir haben noch nie mit jemanden fünf Minuten hier über seine KPIs oder zehn Minuten über die KPIs erzählen lassen. Das ist eindrucksvoll. Das finde ich sehr spannend und sehr interessant, wie ihr das aufgefächert habt. Das ist bestimmt für den einen oder anderen auch vielleicht eine Inspiration, das auch in die Richtung zu entwickeln. Ja, und gleichzeitig Und gleichzeitig kam auch, finde ich, sehr schön heraus, wie schwierig es wird, wenn man jetzt sagt, wir haben eine super Studie und einer unserer Berater hat das in dem Mittagessen als Gesprächsanlass genutzt, worauf dann der Kunde das wieder in sein nächstes Teammeeting reingetragen hat und vielleicht dann der nächste Entscheider oder die Entscheiderin sich das nochmal runtergeladen hat und dann entsteht daraus irgendwie ein Projekt ein halbes Jahr später. Das kannst du nicht tracken. Das kannst du nicht mehr verfolgen. Das ist schwierig. Es ist nicht alles messbar, wenn man in Leadership-Content investiert, gerade auf dieser Qualified-Leads-Ebene, wie du da jetzt auch gesagt hast. Richtig. Also früher konnten wir noch, früher hatten wir noch Printouts, also wirklich Studien ausgedruckt in größerem Stil und dann konntest du immer hier, 500 hast du bestellt und nach drei Monaten sind noch 100 da, dann kannst du so ein bisschen, okay, also da hat jeder, aber du weißt nicht, wo die 400 hingekommen sind, außer du machst natürlich ein Mailing, dann kannst du das natürlich schon tracken. Aber wenn die natürlich, die liegen dann bei uns immer rum und wer es halt was braucht, oh spannend, nehme ich mit, das kannst du nicht tracken. Und trotzdem investiert ihr ja ganz klar und mit einem eigenen Team in diesen Ansatz. Also braucht man auch ein Stück weit, muss man auch überzeugt sein von diesem Ansatz oder woher beruht das? Du hast, ihr habt ein Future Talk, ich glaube, ihr habt auch schon weit über 100, 150 Folgen oder noch viel mehr. Genau, richtig. Und das ist ja schon eine sehr, wirklich ein sehr konsistentes und konstantes Format, das du selbst ja auch mitbetreust. Also Zweifel scheinen da ja noch nicht aufgekommen zu sein. Wie kommt das? Ne, weil da helfen halt doch die KPIs, mit denen man dann manche Sachen messen kann. Also Medien ist noch am leichtesten und da haben wir halt gemerkt, seit wir Content, eigenen Content produzieren, wir haben ja vor elf, zwölf Jahren damit begonnen, ist unsere Media Attention stark hochgegangen. Und wir sehen ja, über was dann über die Leute geschrieben wird und das ist sehr stark von den Studien getrieben. Also da Medien ist ganz klar zu sehen. Und klar, das dauert ein bisschen. Dann machen wir natürlich Brand, also unsere Markenabteilung, die macht natürlich dann auch entsprechende Analysen, wie wird die Marke, die Leute innerhalb der Schweiz wahrgenommen oder halt bei anderen Unternehmen innerhalb von Deutschland und so weiter. Und da sieht man das auch. Und dann haben wir sogenannte Client Feedback Diskussionen, das heißt Kundengespräche nach Projekten. Und da hören wir dann auch ab und zu, ich habe letztendlich das und so gelesen von euch und dadurch hört man es raus und daher ist der Case für uns relativ klar und wurde auch nie großartig bisher angezweifelt. Und die Tatsache, dass ich das ja auf ein Fünfmann aufbauen konnte, hatte ich ja nicht von Anfang an. Zeigt ja auch, dass dies der richtige Weg ist und man sieht es auch in anderen Ländern Gesellschaften, die eine ähnliche Entwicklung hinter sich haben. Also wenn du das so erzählst, dann ist das für mich total logisch. Also ich meine, wir arbeiten ja auch so, ja, also vielleicht auch ein bisschen intuitiv, aber ihr macht das ja schon sehr strukturiert. Eine Frage hätte ich noch. Du hast gesagt, dass du dich gerade intensiv mit dem Thema Generative AI auseinandersetzt und grundsätzlich ist es wahrscheinlich auch ein Thema, was ihr viel auch bearbeitet, jetzt auch als Research-Thema. Meinst du, dass AI also jetzt rein von der Produktion her mal irgendwann auch für Solidarship-Content relevant wird? Oder ist das einfach was, wo man AI nicht für benutzen kann? Also ich muss dich jetzt schocken. Es ist schon relevant. Ja? Es ist jetzt schon relevant. Erzähl. Also ich sehe Generative AI, also JetGBT oder all die anderen Tools, ein bisschen als Co-Pilot und ist, glaube ich, eine ganz gute Beschreibung als Assistent-Co-Pilot, der einfach hilft bei mehreren Themen. Ideenentwicklung, würde ich mal sagen, also es geht schneller, um neue Ideen zu entwickeln. Also oft, wenn man ein bisschen so einen Block hat, wenn man ein neues Thema und da hilft es mir sehr. Dann beim Schreiben, also nicht, dass jetzt JetGBT schon komplett eine Studie schreibt, weil es so zu 50 Prozent auch schon geht, aber, sag ich mal, dann bei der Formulierung einfach. Und wir schreiben ja auch viel auf Englisch. Und wenn du dann nicht in deinen eigenen Muttersprache, aber dann mit JetGBT hilft es schon, den Text zu verbessern, relativ schnell, Formulierung und Unterstützung bei einfachen repetitiven Aufgaben. Also ich würde sagen, aktuell spare ich schon mindestens fünf Stunden die Woche durch den Einsatz von dem Tool und es wird noch viel weiter zunehmen. Jetzt bin ich nur im textbasierten Bereich. Noch dramatischer ist es ja bei Visuals. Und also ich sehe es, wir haben ja auch ein Digital-Team, das Kunden berät, was da alles möglich ist. Und da ist der Einsparfaktor, also um was du schneller bist, zum Beispiel, um irgendwelche Bilder zu erstellen, massiv. Also da sprechen wir davon, wo vorher jemand zwei Monate dran saß, machst du jetzt in zwei Stunden und ist kein Witz. Videos. Wir haben ein eigenes Videoproduktionsteam, aber durch den Einsatz von AI, du sparst massiv. Und Video ist ja auch eine Art von Content, den du erstellen kannst. Was aber wichtig ist, weil viele immer Angst haben, ersetzt mich das und so weiter. Also die Message ist ganz klar. Wer einen Job hat in diesen Bereichen, muss sich möglichst schnell mit dem Thema auseinandersetzen. Keiner wird durch Chat-GPT ersetzt, aber er wird definitiv ersetzt durch eine Person, die Chat-GPT anwenden kann. Definitiv. Also das kann ich so behaupten. Was aber auch wichtig ist, und daher habe ich keine Sorge jetzt für, sagen wir mal, Researcher etc. langfristig, weil auch das Tool, gibt es ja die alte Regel, Rubbish in, Rubbish out. Und das gilt natürlich auch für Chat-GPT. Also wenn ich dann nicht das richtig mache, bekomme ich auch nicht die richtigen Antworten. Und da ist die Kunst, Faktencheck zu betreiben. Und wenn ich jetzt so unsere Rolle als Research, unsere tägliche Rolle ansehe, ich glaube, unsere Hauptrolle ist wirklich, Faktencheck zu machen. Das heißt, die einzelnen Quellen zusammentragen und zu schauen, macht das Sinn, stimmt das überhaupt, ist das die richtige Quelle? Und die Herausforderung steigt natürlich jetzt massiv mit den AI-Tools. Und daher glaube ich, dass Leute wie wir und auch wir zunehmend einen noch höheren Wert haben werden in Unternehmen, weil wir das gelernt haben, jahrelang genau zu unterscheiden, das ist richtig, das ist falsch. Und da kommt jetzt ein bisschen meine Sorge, also ich bin euphorisch bei dem Thema, aber auch die Sorge rein, wie es dann der nächsten Generation mal ergehen wird, die vielleicht mit den Tools aufwächst und es nicht mehr lernt, wirklich dieses Faktenchecken oder das Unterscheiden, was ist gute Qualität, was ist schlechte Qualität? Weil um den guten Text beurteilen zu können, musst du idealerweise auch selber schon guten Text schreiben können, behaupte ich mal. Vor allen Dingen, wenn dann Inhalte kommen und das ist ja noch ein bisschen das Manko bei den aktuellen Tools, dass ich eben nicht sehe, woher die Inhalte, die Informationen kommen. Also so gesehen, generell bin ich sehr optimistisch eingestellt, aber es gibt schon viele Risiken, die man beachten muss, aber es wird den ganzen Bereich und am meisten natürlich Marketing, Marketing-Kommunikation massiv verändern. Sehr, sehr spannend. Hast du auch, finde ich, sehr schön auf den Punkt gebracht und ich würde sagen, Fabian, es ist genau das, wie wir es auch sehen und es ist der Co-Pilot, finde ich, ein sehr schönes Bild, weil es super hilfreich sein kann, aber den eigenen Verstand ausschalten, das wird halt zu keinem so guten Ergebnis führen. Das weiß eigentlich auch jeder. Und trotzdem ist das natürlich die Gefahr so da. Ah, ist schön, dass da wird was automatisch generiert und ich muss nichts mehr tun. Nee, so ist es nicht. Vor allen Dingen in der größeren Firma, dann habe ich ja das Reputationsrisiko sofort. Ich meine, das ist ja und drum sind die großen Firmen jetzt natürlich schon aktuell sehr dahinter, das erst mal ein bisschen zu limitieren und die Leute auch zu klären und weil die wissen genau, was da schieflaufen kann. Ja, es ist kein Prozess, der effizienter machen, aber ich glaube, dieses, das, was wir unter Solidarship verstehen, das muss ja irgendwoher kommen, aus irgendwas Erwachsenem. Jemand muss die Idee haben, dass in diesen Daten was Spannendes drin steckt und das kann die KI nicht für dich machen, sondern das musst du ja selber die Erfahrung haben und dann in der Arbeit unterstützt es einen, finde ich auch sehr interessant, was da jetzt gerade passiert, auch für SEO und ja, deswegen. Ja, also SEO ist ein Riesenthema bei uns natürlich auch und ihr habt es ja in etlichen Episoden schon gesagt, okay, jetzt sind wir ja dauernd in SEO und Google investiert, ja, okay, aber what's next? Also ich habe viele in meinem privaten Umfeld, die kaum mehr Google nutzen. Ich weiß nicht, wie das bei euch ist und das heißt, wie werden wir als die Leute mit uns im Inhalt eigentlich jetzt dann bei JetGBT gefunden, ist ein Riesenthema. Ja, sehr, sehr spannendes Thema, aber das würde ich sagen, das wäre vielleicht nur was, was wir noch mal in einer neuen Folge besprechen, Michael. Heute haben wir wirklich über euren Solidarship-Content-Ansatz gesprochen, mega interessant. Vielen Dank, dass du dir die Zeit genommen hast. Sehr gerne und vielen Dank. Danke dir. Bis dann. Ciao. Untertitel im Auftrag des ZDF für funk, 2017
"""

In [27]:
prompt_dev = f"""
Du bist mein persönlicher Assistent, der mir bei der Tages- und Wochenplanung hilft. Ich übermittle nun meine Notizen und Aufzeichnungen. Es ist in chronologisch umgekehrter Reihenfolge aufgebaut, die neuesten Einträge stehen zu Beginn. Die Tage sind in Markdown-Syntax  mit "# " gekennzeichnet, die Themenabschnitte mit "## ". Markierungen für Aufgaben oder Tags können an beliebigen Stellen in den Notizen auftauchen: Offene Aufgaben sind immer mit "[ ]" markiert, erledigte Aufgaben mit "[x]".
Bitte beantworte alle Folgefragen nur und ausschließlich auf Basis der übermittelten Aufzeichnungen.
Gebe alle Texte 1:1 wieder und erfinde nichts neues.
Erstelle eine Liste aller offenen Aufgaben als kurze bullet-point Liste.

Hier die Notizen:
"""


In [48]:
prompt_dev = f"""
You are in the role of an assistant for an editor.
Shorten the following text. Paraphrase the text, do not invent anything new and keep the original tone and style.
The output format is in Markdown syntax.
Write in German language.

"""
# Create 2-3 subheadings.

# Wie wirkt sich Chunk-Größe auf Zusammenfassungsgröße in % aus?

 Chunkgröße | 500 | 750 | 1000 | 1500 | 2000
-------- | -------- | -------- | -------- | -------- | --------
Klima   | 85%   | 78% | 76% | x | 39%
A12 Model DIff   | 88%   | 85% | 87% | x |  38%
- mit GPT4:  | 67%   | 63% | 55% | 50% |  35%
Deloitte Podcast   | 50%   | 28%   | 21%  | x |  16%
B2B Content Podcast | 63%   | 43%  | 43%  | 19% |  17%
- mit GPT4: | 39%   | 31%  | 28%  | 23% |  10%
Adrenalin   | 89%   | 77%   | 77%  | x |  21%
Inhalt   | Inhalt   | Inhalt   | Inhalt  | Inhalt |  Inhalt

# Rules

GPT4 ist immer ca. 25% kürzer

Default: 1000

Sehr dichte Information (Fachtext): 2000

Gesprochen, Podcast: 500

# Rekursiver Test

 Step | Input words | Chunk-Size | 1 | 2 | 3 | 4 | Output words
-------- | -------- | -------- | -------- | -------- | -------- | -------- | --------  
A12 Model DIff | 7900 |1000   | 88% | 49% | 76% | - | 2500
A12 Model DIff | 7900 |1500   | 58% | 23% | 70% | - |753
Deloitte Podcast | 6600 |1000   | 24% | 81% | 99% | 100% | 1300
Deloitte Podcast | 6600 |500   | 51% | 96% | 77% | 98% | 2497

In [111]:
step1 = execute_project('step1', 'gpt-3.5-turbo', prompt_dev, {}, 'Manual prompt', '', '1000', '', 0, True, folders, input_dev, comment)

step2 = execute_project('step2', 'gpt-3.5-turbo', prompt_dev, {}, 'Manual prompt', '', '1000', '', 0, True, folders, step1[2], comment)

step3 = execute_project('step3', 'gpt-3.5-turbo', prompt_dev, {}, 'Manual prompt', '', '1000', '', 0, True, folders, step2[2], comment)

step4 = execute_project('step4', 'gpt-3.5-turbo', prompt_dev, {}, 'Manual prompt', '', '1000', '', 0, True, folders, step3[2], comment)

Model: gpt-3.5-turbo
Started at: 22:39:02

Processing step 1 of 7
 You are in the role of an assistant for an editor. Shorten the following text. Paraphrase the text, do not invent
anything new and keep the original tone and style.  The output format is in Markdown syntax. Write in German language.
``Herzlich willkommen zum Content-Performance-Podcast mit Fabian Deckert und Benjamin Odani, der Online-Marketing-
Podcast für Fortgeschrittene. Hallo Fabian. Hallo Benjamin. Und hallo Michael. Ja, hallo zusammen. Heute sprechen wir
darüber, wie Deloitte Thought Leadership Content entwickelt und dafür haben wir uns einen super Gast geholt in unserem
Podcast, und zwar den Michael Gramp. Du bist Chefökonom bei Deloitte in der Schweiz, Leiter des Research Teams und eben
auch verantwortlich für diese Thought Leadership Publikation. Vielen Dank, dass du dir die Ze

{'token_usage': {'prompt_tokens': 1644, 'completion_tokens': 228, 'total_tokens': 1872}, 'model_name': 'gpt-3.5-turbo', 'wordcount': 